In [11]:
import requests
import arcpy
import pandas as pd
import os
import numpy as np
import zipfile

# === USER INPUT ===
directory = r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb"
gdb_name = "test.gdb"
fgdb = os.path.join(directory, gdb_name)

# === GET FROM GITHUB "BMSB_Data" 
mn_boundary = "C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb\state_of_minnesota"

In [12]:
# === Download and extract NLCD landcover ===
url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip"
os.makedirs(directory, exist_ok=True)
zip_path = os.path.join(directory, "landcover_2019.zip")

with requests.get(url, stream=True) as r, open(zip_path, "wb") as f:
    for chunk in r.iter_content(chunk_size=8192):
        f.write(chunk)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(directory)


In [13]:
landcover_raw = os.path.join(directory, "NLCD_2019_Land_Cover.tif")

In [14]:
def check_box(in_raster, ymin, ymax, xmin, xmax):
    left = float(arcpy.management.GetRasterProperties(in_raster, "LEFT").getOutput(0))
    right = float(arcpy.management.GetRasterProperties(in_raster,"RIGHT").getOutput(0))
    top = float(arcpy.management.GetRasterProperties(in_raster, "TOP").getOutput(0))
    bottom = float(arcpy.management.GetRasterProperties(in_raster,"BOTTOM").getOutput(0))
    
    print(left, right , top , bottom)
    
    if left < xmin or right > xmax or top > ymax or bottom < ymin:
        print("Raster is not Within Bounding Box")
    else: 
        print("Raster is Within Bounding Box")

check_box(landcover_raw, 4770648.492016879, 5490732.38301064, 133186.3828544069, 789594.190423549)


-112390.0 801260.0 5566390.0 4600900.0
Raster is not Within Bounding Box


In [15]:
clip_path = os.path.join(fgdb, "landcover_clip")

arcpy.management.Clip(
    in_raster= landcover_raw,
    rectangle="189775.332000001 4816305.37 762254.4779 5472427.737",
    out_raster= clip_path, 
    in_template_dataset="state_of_minnesota",
    nodata_value="255",
    clipping_geometry="NONE",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\bmsb\\test.gdb\\landcover_clip'>

In [16]:
reclass_path = os.path.join(directory, "land_reclass5")
arcpy.ddd.Reclassify(
    in_raster=clip_path,
    reclass_field="VALUE",
    remap="11 5;21 4;22 4;23 4;24 4;31 4;41 2;42 2;43 2;52 2;71 2;81 3;82 3;90 1;95 1",
    out_raster=reclass_path,
    missing_values="DATA"
)

ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Contact Esri Technical Support (http://esriurl.com/support) to Report a Bug, and refer to the error help for potential solutions or workarounds.
Failed to execute (Reclassify).


In [ ]:
arcpy.conversion.RasterToGeodatabase(
    Input_Rasters=reclass_path,
    Output_Geodatabase=fgdb,
    Configuration_Keyword=""
)


In [ ]:
resample_path = os.path.join(directory, "resample_land")
arcpy.management.Resample(
    in_raster=reclass_path,
    out_raster=resample_path,
    cell_size="1000 1000",
    resampling_type="NEAR"
)


In [ ]:
point_path = os.path.join(fgdb, "lc_point")
arcpy.conversion.RasterToPoint(
    in_raster=resample_path,
    out_point_features= point_path,
    raster_field="Value"
)


In [ ]:
ctu_path = os.path.join(directory, "ctus_with_id")
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features= ctu_path,
    Output_Geodatabase= fgdb
)

In [ ]:
sw_path = os.path.join(fgdb, "ctus_nlcd_SummarizeWithin")
sw_table_path = os.path.join(fgdb, "ncld_ctu_summary")
arcpy.analysis.SummarizeWithin(
    in_polygons= ctu_path,
    in_sum_features= point_path,
    out_feature_class= sw_path,
    keep_all_polygons="KEEP_ALL",
    sum_fields="grid_code Mean",
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="SQUAREKILOMETERS",
    group_field="grid_code",
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="ADD_PERCENT",
    out_group_table= sw_table_path
)

In [ ]:
#Add landcover field
arcpy.management.AddField(
    in_table=sw_table_path,
    field_name="lc_class",
    field_type="TEXT",
    field_precision=None,
    field_scale=None,
    field_length=255,
    field_alias="",
    field_is_nullable="NULLABLE",
    field_is_required="NON_REQUIRED",
    field_domain=""
)

In [ ]:
#Calculate landcover class field
standalone_table = "ncld_ctu_summary" 

land_cover_dict = {
    1: "Wetlands",
    2: "Forest",
    3: "Agriculture",
    4: "Developed",
    5: "Water"
}

with arcpy.da.UpdateCursor(standalone_table, ["grid_code", "lc_class"]) as cursor:
    for row in cursor:
        grid_code = row[0]
        if grid_code in land_cover_dict:
            row[1] = land_cover_dict[grid_code]
            cursor.updateRow(row)

In [ ]:
# Convert the Summarize Within summary table to df 
summary_table = "ncld_ctu_summary"
fields = ["Join_ID", "lc_class", "PercentCount"]

# Read data into a list and convert to df
data = [row for row in arcpy.da.SearchCursor(summary_table, fields)]
df = pd.DataFrame(data, columns=["Join_ID", "lc_class", "PercentCount"])

# Pivot  table to get one row per Join ID with land cover percentages as columns
df_pivot = df.pivot(index="Join_ID", columns="lc_class", values="PercentCount").reset_index()

# Fill NaN values with 0 
df_pivot = df_pivot.fillna(0)

In [ ]:
# Convert the Summarize Within feature class table to df
ctu_table = "ctus_nlcd_SummarizeWithin"
ctu_fields = ["Join_ID", "UNIQUE_ID"]

# Read data into a list and convert to df
ctu_data = [row for row in arcpy.da.SearchCursor(ctu_table, ctu_fields)]
df_ctu = pd.DataFrame(ctu_data, columns=["Join_ID","Unique ID"])



In [ ]:
# Merge pivoted table with  CTU summary table
df_final = df_ctu.merge(df_pivot, on="Join_ID", how="left")
df_final.rename(columns={np.nan: "Unknown_Land_Cover"}, inplace=True)

In [ ]:
for column in df_final.columns:
    print(f"Column Name: {column}, Type: {df_final[column].dtype}")

In [ ]:
# Define structure for numpy array
dtype_structure = [
    ("Join_ID", "i4"),              
    ("Unique_ID", "i4"),
    ("Unknown_Land_Cover", "f8"),  
    ("Agriculture", "f8"),
    ("Developed", "f8"),
    ("Forest", "f8"),
    ("Water", "f8"),
    ("Wetlands", "f8")
]

numpy_array = np.array(
    [tuple(row) for row in df_final.itertuples(index=False)],
    dtype=dtype_structure
)

In [ ]:
# Save to gdb
output_table = os.path.join(directory,"ctu_landcover_summarytable_last")
arcpy.da.NumPyArrayToTable(df_final.to_records(index=False), output_table)

print("Final table created successfully.")

In [ ]:
#Add Join
arcpy.management.AddJoin(
    in_layer_or_view="ctus_nlcd_SummarizeWithin",
    in_field="UNIQUE_ID",
    join_table="ctu_landcover_summarytable_last",
    join_field="Unique_ID",
    join_type="KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS",
    rebuild_index="NO_REBUILD_INDEX",
    join_operation=""
)

In [ ]:
#export joined fc

finaljoin_path = os.path.join(directory, "landcover_ctu_finaljoin")
arcpy.conversion.ExportFeatures(
    in_features="ctus_nlcd_SummarizeWithin",
    out_features= finaljoin_path,
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='GNIS_FEATU "GNIS_FEATU" true true false 4 Long 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.GNIS_FEATU,-1,-1;FEATURE_NA "FEATURE_NA" true true false 254 Text 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.FEATURE_NA,0,253;CTU_CLASS "CTU_CLASS" true true false 255 Text 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.CTU_CLASS,0,254;COUNTY_GNI "COUNTY_GNI" true true false 4 Long 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.COUNTY_GNI,-1,-1;COUNTY_COD "COUNTY_COD" true true false 2 Text 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.COUNTY_COD,0,1;COUNTY_NAM "COUNTY_NAM" true true false 100 Text 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.COUNTY_NAM,0,99;POPULATION "POPULATION" true true false 4 Long 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.POPULATION,-1,-1;SHAPE_Leng "SHAPE_Leng" true true false 8 Double 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.SHAPE_Leng,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.ORIG_FID,-1,-1;UNIQUE_ID "UNIQUE_ID" true true false 4 Long 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.UNIQUE_ID,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.Shape_Area,-1,-1;mean_grid_code "Mean grid_code" true true false 8 Double 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.mean_grid_code,-1,-1;Point_Count "Count of Points" true true false 4 Long 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.Point_Count,-1,-1;Join_ID "JOIN ID" true true false 4 Long 0 0,First,#,ctus_nlcd_SummarizeWithin,ctus_nlcd_SummarizeWithin.Join_ID,-1,-1;OBJECTID "OBJECTID" false true false 4 Long 0 9,First,#,ctus_nlcd_SummarizeWithin,ctu_landcover_summarytable_last.OBJECTID,-1,-1;Join_ID "Join_ID" true true false 8 BigInteger 0 0,First,#,ctus_nlcd_SummarizeWithin,ctu_landcover_summarytable_last.Join_ID,-1,-1;Unique_ID "Unique_ID" true true false 8 BigInteger 0 0,First,#,ctus_nlcd_SummarizeWithin,ctu_landcover_summarytable_last.Unique_ID,-1,-1;Unknown_Land_Cover "Unknown_Land_Cover" true true false 8 Double 0 0,First,#,ctus_nlcd_SummarizeWithin,ctu_landcover_summarytable_last.Unknown_Land_Cover,-1,-1;Agriculture "Agriculture" true true false 8 Double 0 0,First,#,ctus_nlcd_SummarizeWithin,ctu_landcover_summarytable_last.Agriculture,-1,-1;Developed "Developed" true true false 8 Double 0 0,First,#,ctus_nlcd_SummarizeWithin,ctu_landcover_summarytable_last.Developed,-1,-1;Forest "Forest" true true false 8 Double 0 0,First,#,ctus_nlcd_SummarizeWithin,ctu_landcover_summarytable_last.Forest,-1,-1;Water "Water" true true false 8 Double 0 0,First,#,ctus_nlcd_SummarizeWithin,ctu_landcover_summarytable_last.Water,-1,-1;Wetlands "Wetlands" true true false 8 Double 0 0,First,#,ctus_nlcd_SummarizeWithin,ctu_landcover_summarytable_last.Wetlands,-1,-1',
    sort_field=None
)

In [ ]:
arcpy.management.CalculateField(
    in_table="landcover_ctu_finaljoin",
    field="attractiveness",
    expression="((!Agriculture! * 1) + (!Developed!  * 1) + (!Forest! * 0.4) + (!Water! * 0) + (!Wetlands!  * 0.1))/100",
    expression_type="PYTHON3",
    code_block="",
    field_type="FLOAT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)